<img src="../img/Logo.png" width="200">

# Aprendizaje por Imágenes
# Transferencia de aprendizaje (Modelos pre-entrenados)

### Profesor: Jorge Calvo

## Modelo Vgg16

Imagina que quieres enseñar a una computadora a reconocer diferentes objetos en imágenes, como perros, gatos, coches, etc. Para hacerlo, necesitarías miles y miles de imágenes etiquetadas, es decir, imágenes que ya tienen la información de qué objeto se muestra en cada una.

Entrenar un modelo de reconocimiento de objetos desde cero puede llevar mucho tiempo y recursos computacionales. Aquí es donde entra en juego un modelo preentrenado, como VGG16.

Un modelo preentrenado es un modelo de aprendizaje profundo que ya ha sido entrenado en un conjunto masivo de imágenes. En el caso de VGG16, ha sido entrenado en el conjunto de datos ImageNet (https://www.image-net.org/), que contiene millones de imágenes de diferentes categorías.

Durante el entrenamiento, VGG16 aprendió a reconocer características comunes en las imágenes, como bordes, formas y texturas, y a asociar esas características con las etiquetas de las imágenes. Esto significa que VGG16 ya ha aprendido una representación útil y general de las características visuales en las imágenes.

Cuando utilizas un modelo preentrenado como VGG16, puedes aprovechar ese conocimiento previo para resolver tareas específicas sin tener que entrenar desde cero. En lugar de entrenar todo el modelo, puedes tomar el modelo preentrenado y adaptarlo a tu problema específico.

### Estructura
<img src="../img/vgg.png" width="800">

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt





https://keras.io/api/applications/vgg/#vgg16-function

In [ ]:
# Cargar el modelo preentrenado VGG16
model = VGG16(weights='imagenet')

model.summary()

In [ ]:
plot_model(model, to_file='./red_convolucional.png', show_shapes=True, show_layer_names=True, dpi=96)

In [ ]:
# Cargar y preprocesar una imagen de ejemplo
img_path = '../images/camion.jpg'

#Le damos el tamaño admitido por Vgg16
img = image.load_img(img_path, target_size=(224, 224)) #Cargamos la imagen con el tamaño admitido por vgg16
x = image.img_to_array(img) #Convertimos la imagen en un array
x = np.expand_dims(x, axis=0) #Le añadimos la dimensión del batch
x = preprocess_input(x) #Le pasamos el procesamiento particular de vgg16

print("La imagen tiene la forma ",x.shape)

# Realizar la clasificación de la imagen
predictions = model.predict(x)
#Usamos la propiedad decoded_predictions para obetener la mejor probabilidad
decoded_predictions = decode_predictions(predictions, top=5)[0]

print(decoded_predictions)

In [ ]:
# Imprimir las predicciones
for pred in decoded_predictions:
    print(f"{pred[1]}: {pred[2]*100:.2f}%")

## Las activaciones de cada capa convolucional 

In [ ]:
# Obtener las salidas de las diferentes capas por orden
#layer_outputs = [layer.output for layer in model.layers[1:19]]  # Capas convolucionales hasta block4_pool
#activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)


# Visualizar las salidas de las capas por nombres
selected_layers = ['block1_conv1', 'block1_conv2']
layer_outputs = [model.get_layer(layer_name).output for layer_name in selected_layers]
activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)


activations = activation_model.predict(x)

images_per_row = 8

for layer_name, layer_activation in zip(selected_layers, activations):
    num_features = layer_activation.shape[-1]
    size = layer_activation.shape[1]

    # Crear una cuadrícula de imágenes para visualizar las activaciones
    rows = num_features // images_per_row
    display_grid = np.zeros((size * rows, images_per_row * size))

    for row in range(rows):
        for col in range(images_per_row):
            channel_image = layer_activation[0, :, :, row * images_per_row + col]
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[row * size: (row + 1) * size, col * size: (col + 1) * size] = channel_image

    # Mostrar las activaciones de la capa
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    plt.show()


## Fine-tuning Vgg16

In [ ]:
# Init the VGG model
vgg_conv = VGG16(weights="imagenet", include_top=False, input_shape=(224,224, 3))

In [ ]:
vgg_conv.summary()

Para el proceso de Fine Tuning haremos entrenables las capas **block5_conv** que uniremos a nuestras capas finales de clasificación

In [ ]:
vgg_conv.trainable=True
set_trainable=False

# Freeze all the layers
for layer in vgg_conv.layers[:]:
  if layer.name == "block5_conv1":
     set_trainable=True 
  if set_trainable:
     layer.trainable=True
  else: layer.trainable=False   
 
# Check the trainable status of the individual layers

for layer in vgg_conv.layers:
    print(layer, layer.trainable)

In [ ]:
# Create the model
model = Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
print(model.summary())